In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import re
from time import time

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mathe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
raw = pd.read_csv('https://raw.githubusercontent.com/Matheusadler/Sentiment-Analysis-Application/master/g1_clean.csv')

In [4]:
stop_words = set(stopwords.words("portuguese"))
stop_words.update(['que', 'até', 'esse', 
                    'essa', 'pro', 'pra',
                    'oi', 'lá', 'blá', 'bb', 
                    'bbm', 'abm', 'cbm', 
                    'dbm', 'dos', 
                    'ltda', 'editora']), 

clean_comments = []

In [5]:
for w in range(len(raw.Comment)):
  comment = raw['Comment'].iloc[w]

  # remove special characters and digits
  comment  = re.sub("(\\d|\\W)+|\w*\d\w*"," ",comment )
  comment = ' '.join(s for s in comment.split() if (not any(c.isdigit() for c in s)) and len(s) > 2)
  clean_comments.append(comment)

clean_comments[0:5]

['PRESIDENTE LULA ÉPOCA AFIRMOU APÓS TRAGÉDIA OCORRIDA CLA QUE MÁXIMO ANOS SERIA LANÇADO VLS QUE ATÉ AGORA ISSO NUNCA FOI REALIDADE BASE LANÇAMENTO FOI RECONSTRUÍDA SOMENTE SEJA QUASE ANOS APÓS TRAGÉDIA OCORRIDA CENTRO',
 'PRESIDENTE LULA ÉPOCA AFIRMOU APÓS TRAGÉDIA OCORRIDA CLA QUE MÁXIMO ANOS SERIA LANÇADO VLS QUE ATÉ AGORA ISSO NUNCA FOI REALIDADE BASE LANÇAMENTO FOI RECONSTRUÍDA SOMENTE SEJA QUASE ANOS APÓS TRAGÉDIA OCORRIDA CENTRO',
 'Muito loko esse carro',
 'Laércio Dantas não tração dianteira mas tração nas rodas fundo todas elas estão convencendo que esse sistema melhor para pilotos fim semana que andam pelas cidades trackdays por Nisso Audi está anos luz frente com Quattro Até Ferrari lançou Cansaram tomar pau dos mas largadas Tração traseira mata muita gente pois poucos sabem usá Ainda mais carro for realmente forte Esse caminho era inevitável',
 'Será excelente carro pra quem está acostumado com asiáticos mas Mercedes com tração dianteira não engulo']

In [6]:
# COUNT vectorizer
tf_vectorizer = CountVectorizer(
        min_df = 2,
        max_df = 0.95,
        max_features = 10000,
        stop_words = stop_words, 
        ngram_range = (1,2)
  )

#transform
vec_text = tf_vectorizer.fit_transform(clean_comments)

#returns a list of words.
words = tf_vectorizer.get_feature_names()

print(vec_text.shape)
print(len(words))

(187, 1261)
1261


In [7]:
words[0:10]

['abaixo',
 'abaixo novo',
 'abro',
 'abro pura',
 'absurdo',
 'absurdo carro',
 'absurdos',
 'absurdos impostos',
 'acabou',
 'acabou lançar']

In [8]:
from sklearn.decomposition import LatentDirichletAllocation

In [14]:
def print_top_words(model, feature_names, n_top_words):
  for topic_idx, topic in enumerate(model.components_):
    print("\n--\nTopic #{}: ".format(topic_idx + 1))
    message = ", ".join([feature_names[i]
                          for i in topic.argsort()[:-n_top_words - 1:-1]])
    print(message)
  print()

def display_topics(W, H, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("\n--\nTopic #{}: ".format(topic_idx + 1))
        print(", ".join([feature_names[i]
                for i in topic.argsort()[:-no_top_words - 1:-1]]).upper())
        top_d_idx = np.argsort(W[:,topic_idx])[::-1][0:no_top_documents]
        for d in top_d_idx: 
          doc_data = raw[['Author', 'Comment']].iloc[d]
          print('{} - {} : \t{:.2f}'.format(doc_data[1], doc_data[0], W[d, topic_idx]))

In [15]:
lda = LatentDirichletAllocation(n_components=15, 
                                learning_method='online', # 'online' equivale a minibatch no k-means
                                random_state=0)

t0 = time()

lda.fit(vec_text)
doc_topic_matrix = lda.transform(vec_text)

print("done in %0.3fs." % (time() - t0))

done in 0.363s.


In [17]:
print('Matriz documento-tópicos:' + str(doc_topic_matrix.shape))
print('Matriz tópicos-termos:' + str(lda.components_.shape))

Matriz documento-tópicos:(187, 15)
Matriz tópicos-termos:(15, 1261)


In [18]:
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, 20)


Topics in LDA model:

--
Topic #1: 
prefiro, anda, azera, acho, carro, etc, cometários ofensas, eua, excelente, qualidade, vamos respeitar, comprar azera, vista, motor equipado, accord, pior eua, pago, frente, equipado, testes amigos

--
Topic #2: 
lançado mundialmente, carneiro, reportagem, ano, ano passado, modelo, carneiro diz, leitão, ops, ops carneiro, modelo lançado, lançado, diz, reportagem modelo, diz reportagem, leitão ops, mundialmente, passado, mundialmente ano, aqui

--
Topic #3: 
audi, série, preço, brasil, comprar, bmw, bmw série, motor, pobre, lucro, cla, vou comprar, mim, falta, putz, coisa, compra, preço compra, enterrar, versão pobre

--
Topic #4: 
combina, fuscão, garagem fuscão, combina garagem, garagem, pobre, pobre lixo, lixo msm, lixo, msm, nome, lindo, resultado, total, fiasco total, mundo, estrelinha, cla completa, classe, ungido estrelinha

--
Topic #5: 
povo, governo, povo povo, aqui, voto, sim lado, povo coisa, espaço, facilita sim, governo outra, governo p

In [19]:
display_topics(doc_topic_matrix,
               lda.components_, 
               words,
               raw,
               15, 
               10)


--
Topic #1: 
PREFIRO, ANDA, AZERA, ACHO, CARRO, ETC, COMETÁRIOS OFENSAS, EUA, EXCELENTE, QUALIDADE, VAMOS RESPEITAR, COMPRAR AZERA, VISTA, MOTOR EQUIPADO, ACCORD
Jorge Pessoal! Ponto de vista cada um tem o seu, vamos respeitar os cometários sem ofensas. Tenho uma c180 2012/2012, excelente carro, mecânica incrível e acho que a motorização é "suficiente", são exatamente os mesmos 156 cv com 25,5 kgfm de torque. NÃO é um super-esportivo, e claro que gostaria de ter acesso as c300 ou c350 que são vendidas lá fora, mas na boa, o conjunto da obra é fantástico (conforto + desempenho + qualidade + dirigibilidade). Fiz os testes com amigos e: fusion 2.0 240 cv não anda junto, anda na frente até os 140 km/h dos blocos v6 de SUV e do Accord/Azera/etc. (cont.) - 'Bruno Souza : 	0.99
Jorge Pessoal! Ponto de vista cada um tem o seu, vamos respeitar os cometários sem ofensas. Tenho uma c180 2012/2012, excelente carro, mecânica incrível e acho que a motorização é "suficiente", são exatamente os mesm

In [20]:
main_topic = []
mt_prob = []
for l in range(len(raw['Author'])):
  main_topic.append(doc_topic_matrix[l,:].argmax() + 1)
  mt_prob.append(doc_topic_matrix[l,:].max())

raw = raw.assign(main_topic = main_topic, main_topic_prob = mt_prob)

In [23]:
topico = 13
pd.options.display.max_colwidth = 300
raw[raw['main_topic'] == topico].sort_values('main_topic_prob', ascending = False)[['main_topic_prob','Author', 'Comment']].head(20).sample(10)

,main_topic_prob,Author,Comment
144,0.970833,'Adm 1958,"Mercedes é uma estrela, mas convenhamos 147 mil é um valor muito alto. Fui ver o carro é lindo, mas o porte ficou pequeno, acho que nem para médio chega a ser."
47,0.766666,'Eduardo Gouveia,longitudiAnal se vai ver... kkk
178,0.883333,'Bruno Favoretto,"Isso ai que é difícil Gustavo, ver civic de 124k \'-\'"
30,0.813333,'Fernando Eu,"As Lima , a CLA custa $29.900 aqui nos USA ."
160,0.844444,'Ronam Bastos,"Bom Passat é 179K, o Fusion 150K e o CLA 183K, vai entender os critérios das montadoras para colocarem preços."
177,0.844444,'Gustavo Gonçalves,Pior é quem dá 124 mil em um civic.
42,0.813333,'Farsa Pt,"Não tem nem como comparar os dois,a Mb leva vantagem pelo nome que ela ja fez pelo mundo!!"
185,0.977778,'Caravaggio,"Não gostei da nova traseira da linha, por dentro também ficou limpa demais lembra os carros dos anos 70. O conjunto mecânico + status ainda valem a pena, mas hoje as bmw estão mais bonitas na minha opinião."
122,0.989394,'Tim Rabey,"1.6 e 156 cv por quase 150 conto? Peraí Mercedes, ai já é demais, pra que entrou no Inovar Auto? BMW 320i Sport Top com um 2.0T 184CV é infinitamente superior em todos os aspectos pelo mesmo preço! CLA pelo que tinha lido era para vir em um posicionamento um pouco abaixo do (novo) classe C, mas ..."
167,0.813333,'Rafael Moraes,Não sabia que ele era mais caro que o Classe C. Pensei que esse fosse o sedã de entrada da marca.


In [25]:
pyLDAvis.sklearn.prepare(lda, vec_text, tf_vectorizer, sort_topics=False, mds = 'tsne')

e:\python\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
0      114.344879  114.308296       1        1   8.160844
1      105.821045  -93.749626       2        1   2.779242
2     -138.334473   95.320763       3        1   7.882666
3       -7.436697  111.259697       4        1   0.844797
4     -207.550629 -105.472603       5        1   2.144169
5       52.817650   14.603765       6        1   8.149296
6      168.010849    6.806950       7        1   4.586491
7      -94.169601 -157.789551       8        1   7.575429
8     -122.849174  -39.059418       9        1   9.986669
9      -94.894264  195.408295      10        1   5.281468
10      24.927401 -180.464874      11        1   8.649820
11     -53.413467   26.832108      12        1   9.928481
12     -12.255850  -72.093201      13        1  12.622247
13    -224.217957   28.070581      14        1   2.263732
14      36.154255  209.427582      15        1   9.144650, topic_info=            Term       Freq      Total Category  logprob  loglift
947         povo   5.000000   5.000000  Default  30.0000  30.0000
190        carro  37.000000  37.000000  Default  29.0000  29.0000
1177         vai  17.000000  17.000000  Default  28.0000  28.0000
582      governo   4.000000   4.000000  Default  27.0000  27.0000
90          aqui   7.000000   7.000000  Default  26.0000  26.0000
...          ...        ...        ...      ...      ...      ...
244          cla   2.278198  14.562580  Topic15  -4.9164   0.5369
666      lançado   1.793436   4.384075  Topic15  -5.1557   1.4982
582      governo   1.792190   4.879207  Topic15  -5.1564   1.3905
376        desse   1.784929   7.856676  Topic15  -5.1604   0.9100
170   brasileiro   1.784746   4.754996  Topic15  -5.1605   1.4121

[707 rows x 6 columns], token_table=      Topic      Freq                Term
term                                     
4         7  0.423732             absurdo
5         7  0.425182       absurdo carro
6        14  0.634743            absurdos
7        14  0.632628   absurdos impostos
10        1  0.409730              accord
...     ...       ...                 ...
1252      7  0.548613                ágio
1253      7  0.544789      ágio conhecido
1258     14  0.627131               órgão
1259     14  0.628042  órgão organizações
1260     14  0.640179              ônibus

[763 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])